In [1]:
# Common utilities for web search
from duckduckgo_search import DDGS
from typing import List, Dict
from dotenv import load_dotenv
import os

load_dotenv()

class WebSearchTool:
    def search(self, query: str, num_results: int = 5) -> List[Dict]:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
        return results

In [2]:
# OpenAI Implementation
from openai import OpenAI
import time
import json

class OpenAIResearchAgent:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.web_search = WebSearchTool()

        # Define custom functions for the assistant
        self.tools = [{
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Search the internet for information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        },
                        "num_results": {
                            "type": "integer",
                            "description": "Number of results to return"
                        }
                    },
                    "required": ["query"]
                }
            }
        }]

        # Create an assistant
        self.assistant = self.client.beta.assistants.create(
            name="Research Agent",
            instructions="""You are a research agent that:
                1. Searches the internet for current information
                2. Analyzes and synthesizes found information
                3. Provides detailed summaries with citations
                4. Suggests follow-up research areas
                When searching, break down complex queries into specific searchable terms.""",
            model="gpt-4-turbo",
            # model="gpt-4-turbo-preview",
            tools=self.tools
        )

    def search_internet(self, query: str, num_results: int = 5) -> str:
        results = self.web_search.search(query, num_results)
        return json.dumps(results)

    def start_research(self, query: str) -> str:
        # Create a thread
        thread = self.client.beta.threads.create()

        # Add the initial message
        # message = self.client.beta.threads.messages.create(
        #     thread_id=thread.id,
        #     role="user",
        #     content=query
        # )

        # Run the assistant
        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant.id
        )

        # Handle tool calls
        while run.status in ["queued", "in_progress"]:
            run = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )

            if run.status == "requires_action":
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []

                for tool_call in tool_calls:
                    if tool_call.function.name == "search_internet":
                        args = json.loads(tool_call.function.arguments)
                        output = self.search_internet(args["query"], args.get("num_results", 5))
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": output
                        })

                run = self.client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

            time.sleep(1)

        # Get the final response
        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        return messages.data[0].content[0].text.value

In [3]:
# Anthropic Implementation
from anthropic import Anthropic

class AnthropicResearchAgent:
    def __init__(self, api_key):
        self.client = Anthropic(api_key=api_key)
        self.web_search = WebSearchTool()
        self.memory = []

    def _create_system_prompt(self):
        return """You are a research agent that systematically:
            1. Searches the internet for current information
            2. Analyzes and synthesizes search results
            3. Maintains context across interactions
            4. Provides citations for all findings
            5. Suggests related research directions"""

    def process_query(self, query: str) -> str:
        # First, perform a web search
        search_results = self.web_search.search(query)

        # Create messages with memory and search results
        messages = [{
            "role": "system",
            "content": self._create_system_prompt()
        }] + self.memory + [{
            "role": "user",
            "content": f"""Query: {query}

            Search Results:
            {json.dumps(search_results, indent=2)}

            Please analyze these results and provide a comprehensive response."""
        }]

        # Get response using Claude
        response = self.client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=2000,
            messages=messages,
            tools=[{
                "type": "function",
                "function": {
                    "name": "perform_additional_search",
                    "description": "Perform additional internet search",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string"},
                            "num_results": {"type": "integer"}
                        },
                        "required": ["query"]
                    }
                }
            }]
        )

        # Update memory
        self.memory.append({
            "role": "user",
            "content": query
        })
        self.memory.append({
            "role": "assistant",
            "content": response.content[0].text
        })

        return response.content[0].text

In [4]:
# Google Gemini Implementation
# import google.generativeai as genai
from google import genai
import sqlite3
from datetime import datetime

class GeminiResearchAgent:
    def __init__(self, api_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-pro')
        self.chat = self.model.start_chat(history=[])
        self.web_search = WebSearchTool()
        self.setup_database()
        self.conn = None

    def setup_database(self):
        self.conn = sqlite3.connect('research_memory.db')
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS research_memory
            (timestamp TEXT, topic TEXT, findings TEXT, sources TEXT)
        ''')
        self.conn.commit()

    def store_memory(self, topic: str, findings: str, sources: List[str]):
        timestamp = datetime.now().isoformat()
        self.cursor.execute(
            'INSERT INTO research_memory VALUES (?, ?, ?, ?)',
            (timestamp, topic, findings, json.dumps(sources))
        )
        self.conn.commit()

    def get_related_memories(self, topic: str) -> List[Dict]:
        self.cursor.execute(
            'SELECT findings, sources FROM research_memory WHERE topic LIKE ?',
            (f'%{topic}%',)
        )
        results = []
        for row in self.cursor.fetchall():
            results.append({
                'findings': row[0],
                'sources': json.loads(row[1])
            })
        return results

    def research_topic(self, query: str) -> str:
        # Perform web search
        search_results = self.web_search.search(query)

        # Get related previous research
        related_memories = self.get_related_memories(query)

        # Create context-aware prompt
        context = f"""Research Query: {query}

        New Search Results:
        {json.dumps(search_results, indent=2)}

        Previous Related Research:
        {
            json.dumps(related_memories, indent=2) 
            if related_memories 
            else 'No previous research found.'
        }

        Please analyze this topic considering:
        1. Key findings from search results
        2. Integration with previous research
        3. Practical applications
        4. Future implications

        Provide citations for all information."""

        # Get response from Gemini
        response = self.chat.send_message({
            'contents': [{
                'role': "user",
                'parts': [{'text': context}]
            }],
            'tools': [{
                'name': 'research_analysis',
                'parameters': {
                    'depth': 'comprehensive',
                    'focus': 'technical and practical'
                }
            }]
        })

        # Store the findings with sources
        self.store_memory(
            query,
            response.text,
            [result['link'] for result in search_results]
        )

        return response.text

In [7]:
# Example usage for each agent
def demonstrate_agents():
    # OpenAI example
    openai_agent = OpenAIResearchAgent(api_key=os.environ.get("OPENAI_API_KEY"))
    openai_result = openai_agent.start_research(
        "What are the latest developments in artificial intelligence research?"
    )
    print("OpenAI Agent Results:", openai_result)

    # Anthropic example
    # anthropic_agent = AnthropicResearchAgent(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    # anthropic_result = anthropic_agent.process_query(
    #     "What are the current challenges in quantum computing?"
    # )
    # print("Anthropic Agent Results:", anthropic_result)

    # Gemini example
    # gemini_agent = GeminiResearchAgent(os.environ.get("GOOGLE_GEMINI_API_KEY"))
    # gemini_result = gemini_agent.research_topic(
    #     "How is AI being used in climate change mitigation?"
    # )
    # print("Gemini Agent Results:", gemini_result)

In [8]:
demonstrate_agents()

OpenAI Agent Results: ### Mutual Funds with the Highest Returns in 2023

The information I've gathered highlights specifics about mutual funds that have shown the highest returns in 2023, including performance in different categories such as large cap, mid-cap, and small-cap funds.

1. **Average Annual Returns**:
   - Large Cap Mutual Funds: 16.15%
   - Mid Cap Mutual Funds: 30.77%
   - Small Cap Mutual Funds: 34.29%
   
   These averages indicate a significant performance difference across the spectrum of fund sizes, with smaller cap funds outperforming their larger counterparts. [Source](https://www.hindustantimes.com/business/5-mutual-funds-that-gave-highest-returns-in-2023-mahindra-nippon-hdfc-101704276858101.html)

2. **Top Performers in Each Category**:
   - **Large Cap Mutual Funds**:
     - XYZ Blue Chip Growth: 18.45%
     - ABC Titan Stability: 17.89%
   
   Although specific fund names like "XYZ Blue Chip Growth" and "ABC Titan Stability" are given as top performers, these a